<a href="https://colab.research.google.com/github/frankwillard/StatGPT/blob/main/GPT_2_Champion_Ranking_without_won_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]

## Load In Libraries, Data

In [ ]:
import pandas as pd

import torch
from torch.utils.data import Dataset

from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2ForSequenceClassification, TrainingArguments, Trainer

In [ ]:
team_data = pd.read_csv('nba_team_advanced_data (1).csv')

In [ ]:
team_data.head()

,Year,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,DRB%,DFT/FGA,Arena,Attend.,Playoffs,W/L%,Losing_season,Champion,won_last,won_last_3
0,1990.0,Atlanta Hawks,28.6,41.0,41.0,44.0,38.0,1.02,-0.39,0.64,...,65.5,0.254,Omni Coliseum,573711.0,N,0.500000,N,N,N,N
1,1990.0,Boston Celtics,30.2,52.0,30.0,51.0,31.0,3.99,-0.76,3.23,...,72.1,0.223,Boston Garden,611537.0,Y,0.634146,N,N,N,N
2,1990.0,Charlotte Hornets,25.6,19.0,63.0,21.0,61.0,-7.82,0.81,-7.00,...,67.5,0.269,Charlotte Coliseum,979941.0,N,0.231707,Y,N,N,N
3,1990.0,Chicago Bulls,26.1,55.0,27.0,50.0,32.0,3.26,-0.51,2.74,...,68.1,0.262,Chicago Stadium,752564.0,Y,0.670732,N,N,N,N
4,1990.0,Cleveland Cavaliers,26.1,42.0,40.0,40.0,42.0,-0.30,-0.31,-0.62,...,67.7,0.202,Coliseum at Richfield,695710.0,Y,0.512195,N,N,N,N


In [ ]:
team_data.columns

Index(['Year', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS',
       'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%',
       'ORB%', 'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Arena',
       'Attend.', 'Playoffs', 'W/L%', 'Losing_season', 'Champion', 'won_last',
       'won_last_3'],
      dtype='object')

In [ ]:
# Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship.
df = team_data[["Year", "Team", "Age" , "MOV" , "NRtg" ,"TS%", "W/L%"]]
# df.rename(columns={'PTS_per_game': 'Points', 'TRB_per_game': 'Rebounds', 'AST_per_game': 'Assists', 'STL_per_game': 'Steals', 'BLK_per_game': 'Blocks', "VORP_advanced" : "vorp"}, inplace=True)


# Convert all columns to integer type
df = df.astype(float, errors='ignore')

# cond1 = (df["Points"] < 99) & (df["Rebounds"] < 99) & (df["Assists"] < 99) & (df["Steals"] < 99) & (df["Blocks"] < 99)
# cond2 = (df["Points"] > 0) & (df["Rebounds"] > 0) & (df["Assists"] > 0) & (df["Steals"] > 0) & (df["Blocks"] > 0)

# df = df[cond1]
# df = df[cond2]

df.to_csv('teams_stats.csv', index=False)

#player_data = player_data[cond]


In [ ]:
# all_nba = player_data[["Player", "All_NBA"]]
# all_nba.rename(columns={"All_NBA": 'allnba'}, inplace=True)

# all_nba =  all_nba[all_nba["Player"].isin(df["Player"])]
# all_nba.to_csv('all_nba.csv', index=False)

In [ ]:
# vorp = player_data[["Player", "VORP_advanced"]]
# vorp.rename(columns={"VORP_advanced": 'vorp'}, inplace=True)
# vorp =  vorp[vorp["Player"].isin(df["Player"])]
# vorp.head()
# vorp.to_csv('vorp.csv', index=False)


# vorp['vorp'] = vorp['vorp'].apply(lambda x: 0 if x <= 0 else 1)
# vorp.to_csv('vorp_binary.csv', index=False)

In [ ]:
champion = team_data[["Year", "Team", 'Champion']]

In [ ]:
len(champion)

963

In [ ]:
champion['Champion'] = champion['Champion'].apply(lambda x: 0 if x == 'N' else 1)

<ipython-input-34-fcf44467c76d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champion['Champion'] = champion['Champion'].apply(lambda x: 0 if x == 'N' else 1)


In [ ]:
champion['Champion'].value_counts()

0    930
1     33
Name: Champion, dtype: int64

In [ ]:
champion.to_csv('champion_binary.csv', index=False)

## Define Train/Test Split

In [ ]:
team_stats = pd.read_csv('teams_stats.csv')
# vorp = pd.read_csv('vorp_binary.csv')
champion = pd.read_csv('champion_binary.csv')

In [ ]:
# from sklearn.model_selection import train_test_split
# train_data, test_data, train_labels, test_labels = train_test_split(team_stats, champion, test_size=0.25, random_state=42)

In [ ]:
train_data = team_stats[team_stats['Year'] < 2013]
test_data = team_stats[team_stats['Year'] > 2014]
train_labels = champion[champion['Year'] < 2013]
test_labels = champion[champion['Year'] > 2014]

In [ ]:
train_data.index = range(len(train_data))
train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
test_labels.index = range(len(test_labels))

In [ ]:
len(train_data)

663

In [ ]:
len(train_labels)

663

## Convert Tabular Data to Text Input

In [ ]:
training_inputs = []

# Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship.
for index, row in train_data.iterrows():
  # Will include more in future
  # won_last = 'did not win' if row['won_last'] == 'N' else 'won'
  train_input = f"{row['Team']} in {row['Year']} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {row['W/L%']}"
  training_inputs.append(train_input)

training_labels = []
for _,row in train_labels.iterrows():
  # Will include more in future
  train_label = f"{row['Champion']}"
  training_labels.append(int(train_label))

In [ ]:
# Initialize empty lists for test inputs and labels
testing_inputs = []
testing_labels = []

# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    # won_last = 'did not win' if row['won_last'] == 'N' else 'won'
    test_input = f"{row['Team']} in {row['Year']} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {row['W/L%']}"
    testing_inputs.append(test_input)

# Loop through test_labels for labels
for _, row in test_labels.iterrows():
    # Will include more in the future
    test_label = f"{row['Champion']}"
    testing_labels.append(int(test_label))


In [ ]:
len(training_inputs)

663

# Define Model and Start Training

In [ ]:
model_name = "gpt2-large"  # or other sizes like "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model.cuda()



tokenized_training_data = tokenizer(training_inputs, padding=True)
tokenized_testing_data = tokenizer(testing_inputs, padding=True)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class Dataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.tokenized_data = tokenized_data
        self.labels = labels

    def __len__(self):
        return len(self.tokenized_data["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}

        item["labels"] = torch.tensor(self.labels[idx])

        return item

In [ ]:
# Define training dataset
training_dataset = Dataset(tokenized_training_data, training_labels)
testing_dataset = Dataset(tokenized_testing_data, testing_labels)

In [ ]:
len(training_dataset)

663

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Define Trainer parameters
def compute_metrics(p):

    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "recall": recall, "f1": f1}

# Define training arguments
training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    seed=0
)


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset,
    compute_metrics=compute_metrics,
)

# Start fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,No log,0.141492,0.966667,0.000000,0.000000
2,No log,0.138758,0.966667,0.000000,0.000000
3,No log,0.132593,0.966667,0.000000,0.000000
4,No log,0.131906,0.966667,0.000000,0.000000
5,No log,0.130888,0.966667,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

TrainOutput(global_step=55, training_loss=0.19482394131747158, metrics={'train_runtime': 35.0516, 'train_samples_per_second': 94.575, 'train_steps_per_second': 1.569, 'total_flos': 86281513943040.0, 'train_loss': 0.19482394131747158, 'epoch': 5.0})

In [ ]:
# count = 0
# total = 0
# batch_size = 64

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)


# model = model.cuda()
# for i in range(0, len(testing_inputs), batch_size):
#     sentences = testing_inputs[i:i+batch_size]
#     labels = torch.tensor(testing_labels[i:i+batch_size])
#     t = tokenizer(sentences, return_tensors="pt", padding = True)
#     t = {key: val.cuda() for key, val in t.items()}

#     pred = model.forward(**t).logits.argmax(-1).detach().cpu()


#     count += (pred == labels).sum()


# print(count)
# print(len(testing_inputs))

In [ ]:
count = 0
total = 0
batch_size = 30

TP, FP, TN, FN = 0, 0, 0, 0

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model = model.cuda()

for i in range(0, len(testing_inputs), batch_size):
    sentences = testing_inputs[i:i + batch_size]
    labels = torch.tensor(testing_labels[i:i + batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding=True)
    t = {key: val.cuda() for key, val in t.items()}

    champion_preds = model.forward(**t).logits.detach().cpu()[:,1]
    predicted_champion = champion_preds.argmax()
    print(testing_inputs[i + predicted_champion])

Atlanta Hawks in 2015.0 has average Age: 27.8, MOV: 5.43, NRtg: 5.8, TS: 0.563, Win Percentage: 0.731707317073171
Cleveland Cavaliers in 2016.0 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695121951219512
Toronto Raptors in 2017.0 has average Age: 26.1, MOV: 4.21, NRtg: 4.5, TS: 0.561, Win Percentage: 0.621951219512195
Toronto Raptors in 2018.0 has average Age: 25.8, MOV: 7.78, NRtg: 7.9, TS: 0.575, Win Percentage: 0.719512195121951
Golden State Warriors in 2019.0 has average Age: 28.4, MOV: 6.46, NRtg: 6.4, TS: 0.596, Win Percentage: 0.695121951219512
Los Angeles Lakers in 2020.0 has average Age: 29.5, MOV: 5.79, NRtg: 5.7, TS: 0.573, Win Percentage: 0.732394366197183
Los Angeles Clippers in 2021.0 has average Age: 28.8, MOV: 6.18, NRtg: 6.4, TS: 0.599, Win Percentage: 0.652777777777778
Boston Celtics in 2022.0 has average Age: 26.1, MOV: 7.28, NRtg: 7.5, TS: 0.578, Win Percentage: 0.621951219512195


In [ ]:
import torch.nn.functional as F

In [ ]:
count = 0
total = 0
batch_size = 30

TP, FP, TN, FN = 0, 0, 0, 0

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model = model.cuda()

for i in range(0, len(testing_inputs), batch_size):
    sentences = testing_inputs[i:i + batch_size]
    labels = torch.tensor(testing_labels[i:i + batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding=True)
    t = {key: val.cuda() for key, val in t.items()}

    champion_preds = model.forward(**t).logits.detach().cpu()[:,1]
    champion_probs = F.softmax(champion_preds, dim=0)

    predicted_champion = torch.topk(champion_preds, 3).indices
    predicted_probs = champion_probs[predicted_champion]

    print(testing_inputs[i + predicted_champion[0]])
    print(testing_inputs[i + predicted_champion[1]])
    print(testing_inputs[i + predicted_champion[2]])
    print(predicted_probs)
    print()

Atlanta Hawks in 2015.0 has average Age: 27.8, MOV: 5.43, NRtg: 5.8, TS: 0.563, Win Percentage: 0.731707317073171
Toronto Raptors in 2015.0 has average Age: 26.1, MOV: 3.07, NRtg: 3.3, TS: 0.553, Win Percentage: 0.597560975609756
Washington Wizards in 2015.0 has average Age: 28.6, MOV: 0.72, NRtg: 0.7, TS: 0.534, Win Percentage: 0.560975609756098
tensor([0.0643, 0.0474, 0.0459])

Cleveland Cavaliers in 2016.0 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695121951219512
San Antonio Spurs in 2016.0 has average Age: 30.3, MOV: 10.63, NRtg: 11.3, TS: 0.564, Win Percentage: 0.817073170731707
Golden State Warriors in 2016.0 has average Age: 27.4, MOV: 10.76, NRtg: 10.7, TS: 0.593, Win Percentage: 0.890243902439024
tensor([0.0738, 0.0632, 0.0538])

Toronto Raptors in 2017.0 has average Age: 26.1, MOV: 4.21, NRtg: 4.5, TS: 0.561, Win Percentage: 0.621951219512195
Utah Jazz in 2017.0 has average Age: 26.8, MOV: 3.94, NRtg: 4.3, TS: 0.563, Win Percentage: 0.6219512195